In [85]:
import re

import pandas as pd
import numpy as np
import os

import seaborn as sns


## Concatenate Dataset into one File

In [86]:
from typing import List


def eraseSpace(directory: str):
    files = getFileNameFromDir(direc=directory)
    for file in files:
        realFile = open("{}/{}".format(directory, file), mode="r", encoding="UTF-8")
        data = [f for f in realFile if f.strip()]
        realFile.close()
        realFile = open("{}/{}".format(directory, file), mode="w", encoding="UTF-8")
        realFile.writelines(data)
        realFile.close()


def getFileNameFromDir(direc: str) -> List[str]:
    return [f for f in os.listdir("{}/".format(direc))]


def myConcat(direcToScan: str, outputFilename: str, outputDirec: str = "all_data"):
    files = getFileNameFromDir(direc=direcToScan)
    datasets = pd.concat([pd.read_csv("{}/{}".format(direcToScan, f)) for f in files], axis=0, ignore_index=True)

    datasets.to_csv("{}/{}.csv".format(outputDirec, outputFilename), index=False)

In [87]:
## Concatenate All Dataset into one File
myConcat(direcToScan="datasets", outputFilename="all_data_dataset")

## Concatenate All Entity-tag Dataset into one File
myConcat(direcToScan="entity-tag", outputFilename="all_entity_dataset")

## Concatenate All Segmentation Dataset into one File
myConcat(direcToScan="segments", outputFilename="all_segment_dataset")

## Concatenate All Segmentation Dataset into one File
myConcat(direcToScan="POS/", outputFilename="all_pos_dataset")

## Erase blank from files

In [88]:
try:
    eraseSpace(directory="datasets")
    eraseSpace(directory="entity-tag")
    eraseSpace(directory="POS")
    eraseSpace(directory="segments")
    print("Success")
except:
    print("fail")


Success


## Compare Ner length, POS length, and SEG Length

In [89]:
allEntityDataset = pd.read_csv("all_data/all_entity_dataset.csv")
allPosDataset = pd.read_csv("all_data/all_pos_dataset.csv")
allSegDataset = pd.read_csv("all_data/all_segment_dataset.csv")

# error = [1 for seg, entity, ent, pos in
#          zip(allSegDataset.items(), allEntityDataset["Question"].items(), allPosDataset["Question"].items()) if
#          len(seg) != len(ent) != len(pos)]
myDict = {}
indexs = allEntityDataset.index.tolist()
segs = allSegDataset["Question"].to_list()
tags = allSegDataset["Intent"].to_list()
ents = allEntityDataset["Question"].to_list()
poss = allPosDataset["Question"].to_list()

for seg, tag, ent, pos, index in zip(segs, tags, ents, poss, indexs):
    if len(str(seg).split()) is not len(str(pos).split()) is not len(str(ent).split()):
        myDict.setdefault("index", []).append(index)
        myDict.setdefault("intent", []).append(tag)
        myDict.setdefault("length_seg", []).append(len(str(seg).split()))
        myDict.setdefault("length_pos", []).append(len(str(pos).split()))
        myDict.setdefault("length_ent", []).append(len(str(ent).split()))
        myDict.setdefault("seg", []).append(seg)
        myDict.setdefault("pos", []).append(pos)
        myDict.setdefault("ent", []).append(ent)

print("length_seg: ", len(myDict["length_seg"]))
print("length_pos: ", len(myDict["length_pos"]))
print("length_ent: ", len(myDict["length_ent"]))
print("seg: ", len(myDict["seg"]))
print("pos: ", len(myDict["pos"]))
print("ent: ", len(myDict["ent"]))

# errorPd = pd.DataFrame.from_dict(myDict)
# # errorPd = errorPd.transpose()
# errorPd.to_csv("all_data/diff_length_sentences.csv")
# errorPd
print(myDict)

length_seg:  487
length_pos:  487
length_ent:  487
seg:  487
pos:  487
ent:  487
{'index': [462, 463, 464, 465, 466, 468, 469, 470, 471, 474, 475, 478, 479, 482, 483, 487, 488, 489, 490, 491, 494, 495, 498, 499, 502, 503, 506, 507, 510, 511, 514, 515, 518, 519, 522, 523, 527, 528, 530, 531, 534, 535, 538, 539, 543, 544, 545, 546, 547, 551, 552, 553, 554, 555, 559, 560, 561, 562, 563, 567, 568, 569, 570, 571, 575, 576, 577, 578, 579, 583, 584, 585, 586, 587, 590, 591, 594, 595, 599, 600, 602, 603, 607, 608, 609, 610, 611, 615, 616, 617, 618, 619, 622, 623, 626, 627, 630, 631, 632, 634, 635, 640, 642, 646, 648, 652, 654, 658, 660, 664, 666, 670, 672, 674, 676, 678, 680, 681, 682, 692, 693, 694, 695, 696, 697, 704, 705, 706, 716, 717, 718, 719, 720, 721, 4127, 4129, 4131, 4132, 4134, 4135, 4136, 4137, 4139, 4141, 4142, 4144, 4145, 4146, 4161, 4165, 4176, 4178, 4179, 4180, 4181, 4182, 4183, 4184, 4185, 4207, 4209, 4211, 4212, 4214, 4215, 4216, 4217, 4219, 4221, 4222, 4224, 4225, 4226, 4232

## Find Amount of Sentences per intent as well as Total

In [90]:
## Find Amount of Sentences
allDataset = pd.read_csv("all_data/all_data_dataset.csv")
allDataset.head()

,Question,Intent
0,តើប្រាប់ពីតម្លៃសិក្សារបស់សាលាស៉ីអេឌីធី,NaN
1,តើប្រាប់ពីតម្លៃសិក្សារបស់សាលាCADT,NaN
2,តើខ្ញុំចង់ដឹងពីតម្លៃពីតម្លៃសិក្សារបស់សាលាស៉ីអេ...,NaN
3,តើខ្ញុំចង់ដឹងពីតម្លៃពីតម្លៃសិក្សារបស់សាលាCADT,NaN
4,តើសូមប្រាប់ខ្ញុំពីតម្លៃសិក្សារបស់សាលាស៉ីអេឌីធី,NaN


In [105]:
sns.set(style="whitegrid",color_codes=True)

dataCount = allDataset.groupby(allDataset["Intent"]).size

print(dataCount)
# pal = sns.color_palette("Greens_d", len(dataCount))
# print(dataCount.size)
# rank = dataCount.size.argsort()
# proper = rank.argsort()
# print("datacount: ", dataCount)
# print("rank: ",rank)
# print("proper: ",proper)


<bound method GroupBy.size of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x000001F2F80EE8C8>>

In [92]:
# Find Total
print("Total Sentence length: ", allDataset.Question.count())

Total Sentence length:  7985


## Check amount of entities as well as for any errors

In [93]:
entityDataFrame = pd.read_csv("all_data/all_entity_dataset.csv")
entityFrame = pd.read_csv("Named Entity.csv")

In [94]:
entityFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Named Entity  15 non-null     object
 1   NE Code       15 non-null     object
 2   Remarks       14 non-null     object
dtypes: object(3)
memory usage: 488.0+ bytes


In [95]:
# Remove white space from column name and apply it to csv file
# entityFrame = entityFrame.rename(columns={entityFrame.columns.values[1]: "Question"})
# entityFrame.to_csv("Named Entity.csv")

In [96]:
myDict = {}
entityTag = entityFrame["NE Code"].to_list()
entitySentences = entityDataFrame["Question"].to_list()

for x in entityTag:
    count = 0
    for j in entitySentences:
        sentence = j.split()
        for c in range(len(sentence)):
            # Matches /B-Word until space
            if re.search(r"/B-{}\w*".format(x), sentence[c]):
                count += 1
                split = sentence[c].rsplit("/B-", maxsplit=1)[1]
                myDict.update({split: count})

entityAmount = pd.DataFrame(myDict.items(), columns=["Intent", "Amount"])
# entityAmount
myDict

KeyError: 'NE Code'

In [ ]:
## Calculate the percentage

# Sum of Intent
totalIntentTag = sum(myDict[x] for x in myDict)
rise = ["{0:.2f}%".format(myDict[x] * 100 / totalIntentTag) for x in myDict]

# array = np.asarray(rise)
#
# arrayV2 = np.fromiter(("{0:.2f}%".format(myDict[x]*100/totalIntentTag) for x in myDict), dtype=str)

entityAmount["Percentage"] = rise
entityAmount

## Count each word occurence

## Find Different sentence length

In [ ]:
segmentData = pd.read_csv("all_data/all_segment_dataset.csv")
posData = pd.read_csv("all_data/all_pos_dataset.csv")
posData.drop(columns=["Intent"], inplace=True)
posData.rename(columns={"Intents": "Intent"}, inplace=True)

different_length = [1 if len(str(input)) != len(str(output)) else 0 for input, output in
                    zip(segmentData["Question"].tolist(), posData["Question"].tolist())]
print("{} sentences have different length.".format(sum(different_length)))

In [ ]:
mydict = {}
mydict.update({"intent": 1})